In [ ]:
import pydicom
import numpy as np
import pandas as pd

In [ ]:
import cv2

# Load the CT image
ct_image = cv2.imread("ct_image.jpg")

# Convert the image to grayscale
gray_image = cv2.cvtColor(ct_image, cv2.COLOR_BGR2GRAY)

# Threshold the image to create a binary image of the ROI
ret, threshold_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

# Find the contours in the binary image
contours, _ = cv2.findContours(threshold_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Find the bounding box for the largest contour
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)

# Draw the bounding box on the original image
ct_image = cv2.rectangle(ct_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Show the image with the bounding box
cv2.imshow("CT Image with Bounding Box", ct_image)
cv2.waitKey(0)


In [ ]:


def get_roi_contour(dicom_path, roi_number):
    # Read the DICOM file
    ds = pydicom.dcmread(dicom_path)

    # Get the ROI Contour Sequence
    roi_contours = ds.ROIContourSequence

    # Iterate through the ROI Contours and find the one with the matching ROI Number
    for item in roi_contours:
        if item.ReferencedROINumber == roi_number:
            # Get the Contour Sequence for the matching ROI
            contour_sequence = item.ContourSequence

            # Iterate through the Contour Sequence and print the coordinates
            for contour in contour_sequence:
                print("Contour Data:", contour.ContourData)
                print("Number of Points:", contour.NumberOfContourPoints)

# Example usage
get_roi_contour('path/to/dicom/file.dcm', 1)


In [ ]:


def get_structure_set(dicom_path):
    # Read the DICOM file
    ds = pydicom.dcmread(dicom_path)

    # Get the structure set sequence
    structure_set = ds.StructureSetROISequence

    # Iterate through the structure set and print the information
    for item in structure_set:
        print("ROI Number:", item.ROINumber)
        print("ROI Name:", item.ROIName)
        print("ROI Description:", item.ROIDescription)

# Example usage
get_structure_set('path/to/dicom/file.dcm')


In [ ]:
import numpy as np

def GetContourPixelData(pixlut, contour, prone = False, feetfirst = False):
    pixeldata = []
    pixelX = []
    pixelY = []
    contour = np.array(contour)
    
    x_lut = np.array(pixlut[0])
    y_lut = np.array(pixlut[1])
    
    xv, yv = np.searchsorted(x_lut, contour[:,0]), np.searchsorted(y_lut, contour[:,1])
    
    if prone:
        yv = len(y_lut) - yv
    if feetfirst or prone:
        xv = len(x_lut) - xv
    
    pixeldata = np.column_stack((xv, yv))
    pixelX, pixelY = xv, yv
    
    return pixeldata


In [ ]:
import pydicom

def get_pixlut_from_dicom(dicom_file):
    # Open the DICOM file
    ds = pydicom.dcmread(dicom_file)

    # Get the image position and orientation
    image_position = ds.ImagePositionPatient
    image_orientation = ds.ImageOrientationPatient

    # Create the LUT
    pixlut = [image_position, image_orientation]
    return pixlut


## Creating class


In [ ]:
import pydicom

class DicomContour:
    def __init__(self, dicom_file):
        self.dicom_file = dicom_file
        self.pixlut = self.get_pixlut_from_dicom()
    
    def get_pixlut_from_dicom(self):
        # Open the DICOM file
        ds = pydicom.dcmread(self.dicom_file)

        # Get the image position and orientation
        image_position = ds.ImagePositionPatient
        image_orientation = ds.ImageOrientationPatient

        # Create the LUT
        pixlut = [image_position, image_orientation]
        return pixlut

    def GetContourPixelData(self, contour, prone = False, feetfirst = False):
        # Convert structure data into pixel data using the patient to pixel LUT.
        pixeldata = []
        pixelX = []
        pixelY = []
        # For each point in the structure data
        # look up the value in the LUT and find the corresponding pixel pair
        for p, point in enumerate(contour):
            for xv, xval in enumerate(self.pixlut[0]):
                if (xval > point[0] and not prone and not feetfirst):
                    break
                elif (xval < point[0]):
                    if feetfirst or prone:
                        break
            for yv, yval in enumerate(self.pixlut[1]):
                if (yval > point[1] and not prone):
                    break
                elif (yval < point[1] and prone):
                    break
            pixeldata.append([xv, yv])
            pixelX.append(xv) #linePlot
            pixelY.append(yv) #lineplot

        return pixeldata


## Implenating 

In [ ]:
import os
from DicomConverter import DicomConverter

# Define the directory containing the CT files
ct_dir = 'path/to/ct_files'

# Iterate through the directory
for filename in os.listdir(ct_dir):
    if filename.endswith('.dcm'):
        # Create an object of the DicomConverter class and pass the file path as an argument
        dc = DicomConverter(os.path.join(ct_dir, filename))
        
        # Extract the pixel data from the DICOM file using pydicom
        pixlut = dc.get_pixlut()
        
        # Get the contour pixel data
        contour_data = dc.GetContourPixelData(pixlut)
        
        # Save the contour data to a file or do something else with it
        with open('path/to/output/'+ filename+'.txt','w') as f:
            for item in contour_data:
                f.write("%s\n" % item)



In [ ]:
import numpy as np
import pydicom
from matplotlib import pyplot as plt

def crop_and_bounding_box(dicom_file_path):
    # Read the DICOM file
    ds = pydicom.dcmread(dicom_file_path)
    # Get the pixel data as a numpy array
    pixel_data = ds.pixel_array

    # Get the minimum and maximum x and y coordinates of the pixels
    min_x, max_x = np.min(np.where(pixel_data > 0)[1]), np.max(np.where(pixel_data > 0)[1])
    min_y, max_y = np.min(np.where(pixel_data > 0)[0]), np.max(np.where(pixel_data > 0)[0])
    # Crop the image
    cropped_image = pixel_data[min_y:max_y, min_x:max_x]

    # Create the bounding box
    bounding_box = [(min_y, min_x), (max_y, max_x)]

    return cropped_image, bounding_box


## Machine learnig model 
### Data processing


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pydicom

class CTPreprocessing:
    def __init__(self, data_path):
        self.data_path = data_path

    def load_data(self):
        # Load dicom images
        dicom_images = [pydicom.dcmread(self.data_path + '/' + f) for f in os.listdir(self.data_path) if f.endswith('.dcm')]

        # Extract image data and store it in a list
        image_data = []
        for d in dicom_images:
            image_data.append(d.pixel_array)

        return image_data

    def crop_data(self, image_data, roi_coordinates):
        # Crop images based on the ROI coordinates
        cropped_images = []
        for image in image_data:
            x1, y1, x2, y2 = roi_coordinates
            cropped_images.append(image[y1:y2, x1:x2])

        return cropped_images
    
    
    def get_bounding_box(self, image):
        # Threshold the image to create a binary mask
        thresh = image > np.mean(image)
        # Label the connected regions in the binary mask
        labeled_image = label(thresh)
        regions = regionprops(labeled_image)
        
        # Find the region with the largest area
        max_area = 0
        for region in regions:
            if region.area > max_area:
                max_area = region.area
                max_region = region

        # Get the bounding box coordinates
        minr, minc, maxr, maxc = max_region.bbox
        bounding_box_coordinates = (minr, minc, maxr, maxc)
        
        return bounding_box_coordinates

    def create_bounding_box(self, image_data, bounding_box_coordinates):
        # Create a bounding box around the ROI
        bounding_box_images = []
        for image in image_data:
            x1, y1, x2, y2 = bounding_box_coordinates
            bounding_box_images.append(image[y1:y2, x1:x2])

        return bounding_box_images

    def preprocess_data(self, image_data):
        # Scale the image data
        scaler = StandardScaler()
        image_data = scaler.fit_transform(image_data)

        # Shuffle the data
        image_data, labels = shuffle(image_data, labels, random_state=42)

        # Split data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

        return X_train, X_test, y_train, y_test


ct_preprocessing = CTPreprocessing(data_path)
image_data = ct_preprocessing.load_data()
cropped_images = ct_preprocessing.crop_data(image_data, roi_coordinates)
bounding_box_images = ct_preprocessing.create_bounding_box(cropped_images)
    
    
    
    
    
# Import necessary libraries
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# Define the input layer
inputs = Input((256, 256, 1))

# Define the encoder layers
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

# Define the decoder layers
up1 = UpSampling2D(size=(2, 2))(pool3)
conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
up2 = UpSampling2D(size=(2, 2))(conv4)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up2)
up3 = UpSampling2D(size=(2, 2))(conv5)
conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(up3)

# Define the output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv6)

# Create the model
model = Model(inputs=[inputs], outputs=[outputs])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on your labeled CT images dataset
model.fit(x_train, y_train, epochs=50, batch_size=32)

# Use the trained model to predict the ROI in a new CT image
prediction = model.predict(x_test)
    

In [ ]:
## anamoly dtection

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Create the data generator
data_gen = ImageDataGenerator(rescale=1./255)

# Create the data flow
train_gen = data_gen.flow_from_directory(
    'path/to/data/train',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)
val_gen = data_gen.flow_from_directory(
    'path/to/data/val',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

# Train the model
model.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=10, validation_data=val_gen)



In [ ]:
import pydicom

# Load DICOM file
ct_image = pydicom.dcmread("path/to/ct_image.dcm")

# Extract image data and metadata
image_data = ct_image.pixel_array
metadata = ct_image.dir()


import cv2

# Resample image
resampled_image = cv2.resize(image_data, (256, 256))

# Normalize image
normalized_image = cv2.normalize(resampled_image, None, 0, 1, cv2.NORM_MINMAX)

# Crop image
cropped_image = normalized_image[50:200, 50:200]


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Use model to predict on new image
prediction = model.predict(new_image)

# Analyze prediction to determine if anomaly is present
if prediction > 0.5:
    print("Anomaly detected")
else:
    print("No anomaly detected")

